# Chapter 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install nltk
!pip install pandas
!pip install tensorflow

In [6]:
path="/content/drive/MyDrive/Colab Notebooks"

In [7]:
import os
os.chdir(path)

In [4]:
import pandas as pd

In [8]:
X_train_w=pd.read_csv("X_train_Imdb.csv",index_col=False)
X_test_w=pd.read_csv("X_test_Imdb.csv",index_col=False)
y_train=pd.read_csv("y_train_Imdb.csv",index_col=False)
y_test=pd.read_csv("y_test_Imdb.csv",index_col=False)
X_train_d=pd.read_csv("X_train_coded_Imdb.csv",index_col=False)
X_test_d=pd.read_csv("X_test_coded_Imdb.csv",index_col=False)

In [11]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download("wordnet")
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()


# Found this handy function online but also lost the source where I found it. I slightly modified it to remove the padding automatically.

################################################################3
# Handy function for lemmatizing and cleaning the text
#################################################################

def clean_text(text):
    #text=text[]
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

################################################################3
# End of Function
#################################################################

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#X_train_w["0"]

In [ ]:
#X_train_w["0"].apply(clean_text)

In [12]:
X_train_w=X_train_w["0"].apply(clean_text)
X_test_w=X_test_w["0"].apply(clean_text)

In [13]:
X_train_w.to_numpy()

array(['scream able filmmaker series see 10 part br screen sure wa one life always life good huge look brilliant wa cinema decide major history br alone performance never model slut major saw film funny pretty person',
       'wa doe accent br look give wang weird win lee superb br screen hang write borrow self wa play else rule 1997 modern would 1996 play version script le br hear good spoiler late consider slasher think sense build sword davis b dog film beach br fair view',
       'arent badly ha werent bruce like brave dreadful find seem accidentally le good br excellent instead around guy time somewhat unwatchable love france smallest look make enjoy rest perhaps badly play wife consist downright get',
       ...,
       'wa one point mostly completely br ariel act hot wa say ensemble experience wa sexual episode wa due br tell ill br fast always life see cast run x br one roommate opinion may point pop lot br american ever',
       'funny child spoil fan element roll instead vanc

In [14]:
tokenizer_training=pd.concat([X_train_w,X_test_w],ignore_index=True)
tokenizer_training


0       scream able filmmaker series see 10 part br sc...
1       wa doe accent br look give wang weird win lee ...
2       arent badly ha werent bruce like brave dreadfu...
3       might sexuality since find even actor weird ke...
4       wa least new complain church dvd ha give scale...
                              ...                        
5731    poor chase good decency usual upon surely stuf...
5732    give ha never lie movie kid br enough always l...
5733    joke ridiculous leader br headache unfair movi...
5734    wa body character later wa puppet character wa...
5735    ive angst candidate deliver get work number ne...
Name: 0, Length: 5736, dtype: object

In [15]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
vocabSize=10000 #We are using only most frequent 10000 words.
embed_dim = 64
lstm_out = 32
embed_size=128 # To Change
max_features = 6000
#
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(tokenizer_training)

In [26]:
# splitting our test set into testing and validation pieces
X_valid,X_test=X_test_w[:350],X_test_w[350:]
y_valid,y_test=y_test[:350],y_test[350:]

In [27]:
X_train = tokenizer.texts_to_sequences(X_train_w)
X_valid = tokenizer.texts_to_sequences(X_valid)
X_test = tokenizer.texts_to_sequences(X_test)
# We will tokenize the test set later on
#
maxlen = 130

In [28]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = 30
X_train = pad_sequences(X_train,maxlen=maxlen)
X_valid = pad_sequences(X_valid,maxlen=maxlen)
X_test = pad_sequences(X_test,maxlen=maxlen)

In [36]:
y_test=y_test["0"]
y_train=y_train["0"]
y_valid=y_valid["0"]

In [29]:
X_train.shape,X_test.shape,X_valid.shape


((2963, 30), (2423, 30), (350, 30))

In [32]:
from keras.layers import Dense , Input , Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D

model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, return_sequences = True))
#model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(LSTM(32, return_sequences = True))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="softmax"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         768000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, None, 128)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 64)          49408     
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 32)          12416     
_________________________________________________________________
module_wrapper (ModuleWrappe (None, 32)                0         
_________________________________________________________________
module_wrapper_1 (ModuleWrap (None, 20)                660       
_________________________________________________________________
module_wrapper_2 (ModuleWrap (None, 20)               

**Please Note**
I tried my best to include Bidirectional Layer. But it is causing issue with higher versions, incompatible with lower versions and is causing issues with everything. I couldn't don't anything about it. 

In [39]:
batch_size = 60
epochs = 25
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_valid,y_valid))

Epoch 1/25
50/50 [==============================] - 4s 75ms/step - loss: 0.0378 - accuracy: 0.5413 - val_loss: 1.0161 - val_accuracy: 0.5771
Epoch 2/25
50/50 [==============================] - 3s 58ms/step - loss: 0.0182 - accuracy: 0.5413 - val_loss: 1.1694 - val_accuracy: 0.5771
Epoch 3/25
50/50 [==============================] - 3s 58ms/step - loss: 0.0127 - accuracy: 0.5413 - val_loss: 1.2658 - val_accuracy: 0.5771
Epoch 4/25
50/50 [==============================] - 3s 58ms/step - loss: 0.0092 - accuracy: 0.5413 - val_loss: 1.4458 - val_accuracy: 0.5771
Epoch 5/25
50/50 [==============================] - 3s 58ms/step - loss: 0.0069 - accuracy: 0.5413 - val_loss: 1.5586 - val_accuracy: 0.5771
Epoch 6/25
50/50 [==============================] - 3s 57ms/step - loss: 0.0073 - accuracy: 0.5413 - val_loss: 1.6136 - val_accuracy: 0.5771
Epoch 7/25
50/50 [==============================] - 3s 58ms/step - loss: 0.0060 - accuracy: 0.5413 - val_loss: 1.6408 - val_accuracy: 0.5771
Epoch 8/25
50

In [40]:
y_pred=model.predict(X_test)

In [42]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1088
           1       0.55      1.00      0.71      1335

    accuracy                           0.55      2423
   macro avg       0.28      0.50      0.36      2423
weighted avg       0.30      0.55      0.39      2423



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Final Conclusions**

We can definitely improve this model, with proper test train split to avoid any Bias. But, the errors and debugging took so much time that I am finally submitting only the working version of my project.

Sentiments identifying is a tough job. Especially since a lot of these seem to have some kind of sarcastic tone, sometimes they aren't even properly placed. All the appropriate steps have been undertaken and addressed with here. In future, when all the errors and incompatibilities have been eliminated, we will probably see a lot more clear version with this.